--------

# 解决思维谜题：“马赛克”

马赛克 是一种类似扫雷的变体逻辑谜题.

- 你需要涂黑一些方格. 
- 数字表示包括它所在方格在内及周围一共九个方格内涂黑的方格数. 


> 注意这个和数独不大相同，因为可能存在数字为0的情况（九宫格没有涂黑的元素）。

In [1]:
from __future__ import print_function
from ortools.sat.python import cp_model as cp
def GetNinePossibleNeighbours(X, Y, pos):
    
    neighbours = []
    directions = [-1, 0, 1]
    
    for ydir in directions:
        for xdir in directions:
            if (pos[0] + ydir >= Y or pos[0] + ydir < 0) or (pos[1] + xdir >= X or pos[1] + xdir < 0):
                continue
            neighbours.append((pos[0] + ydir, pos[1] + xdir))
    
    return neighbours

def MosaicSolver(X, Y, grid):
    print(len(grid))
    if len(grid) != X*Y:
        raise Exception("Grid 的网格和XY不符")

    model = cp.CpModel()
    x = {}
    for i in range(Y):
        for j in range(X):
            x[i, j] = model.NewBoolVar(name = f"x[{i}, {j}]")
    
    for idx, num in enumerate(grid):
        
        if num not in "0123456789*":
            raise Exception("检查输入格式, 只允许输入字符, 0123456789*")
        
        if num in "0123456789":
            x_idx , y_idx = idx % X, idx // Y
            neighbours = GetNinePossibleNeighbours(X, Y, (y_idx, x_idx))
            cage = [x[i[0], i[1]] for i in neighbours]
            model.Add(sum(cage) == int(num))
    
    solver = cp.CpSolver()
    status = solver.Solve(model)
    
    if status == cp.OPTIMAL:
        for i in range(Y):
            for j in range(X):
                print(solver.Value(x[i, j]), end=" ")
            print()
        print()
        
        print("NumConflicts:", solver.NumConflicts())
        print("NumBranches:", solver.NumBranches())
        print("WallTime:", solver.WallTime())
    
    else:
        print("Can't find Optimal.")
    
if __name__ == "__main__":
    iptStr = "222*3**33***5**4****34*****564447874224*2*245**5*3*36*6*245*3*46*****5*5**43456***58**56*****4*6*76*25**77*4**86***6655**5*8**455**54*44455***45**334**53*4**343244*334******54*455**3**22*24***3*4*4*****43**4*454**6*33*4*223********4*****5****357*6****4*666454*345*7****444**5*775**45*7**64*2355**565*1445464*****7***545*0****5*5***6***5*46**24**4*33**56*6*546****586543**434***56**3***532***1*332123*"
    
    MosaicSolver(20,20, iptStr)

    

400
1 1 0 1 0 0 1 1 0 1 0 1 1 1 1 1 0 0 0 1 
0 0 0 0 1 1 1 0 0 1 0 0 1 1 1 1 0 0 1 0 
1 0 1 0 0 1 0 1 1 1 0 0 1 1 0 0 0 0 1 1 
1 0 0 1 1 0 1 1 0 0 0 1 0 1 1 0 0 1 1 0 
1 0 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 0 1 
0 0 1 1 1 0 1 0 1 1 0 1 0 0 1 1 1 1 1 0 
0 1 1 0 1 1 0 0 0 1 1 1 0 0 1 0 0 0 0 1 
1 0 1 1 1 0 1 1 1 0 1 0 1 1 0 0 0 1 1 0 
0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 0 
0 1 1 0 0 1 0 1 0 1 1 0 1 1 0 0 1 1 0 1 
0 1 0 1 0 0 1 0 0 1 0 0 0 1 0 0 1 0 1 0 
0 0 0 0 1 1 0 0 1 0 1 0 1 1 1 0 0 0 1 0 
1 0 0 1 1 0 1 1 0 1 1 1 0 0 1 1 1 1 0 1 
0 1 0 1 1 1 1 1 1 0 0 0 0 1 1 0 1 0 1 0 
0 1 1 0 1 1 0 0 1 0 0 1 1 0 0 1 1 1 1 0 
0 0 1 0 1 1 1 0 1 0 0 1 0 1 1 0 0 1 0 1 
0 0 1 0 0 0 1 0 0 1 0 1 1 1 1 1 0 0 0 1 
0 0 0 1 0 1 0 1 1 0 1 1 1 0 0 1 1 1 1 1 
0 1 0 1 1 1 0 0 0 0 0 0 0 1 1 0 0 1 0 1 
1 1 0 1 1 1 1 0 1 0 1 0 0 0 1 0 0 0 1 0 

NumConflicts: 1
NumBranches: 190
WallTime: 0.016482


![](../assets/figures/Mosaic.png)

![](../assets/figures/Mosaic2.png)